To run this, press "*Runtime*" and press "*Run all*" on a **free** Tesla T4 Google Colab instance!
<div class="align-center">
<a href="https://unsloth.ai/"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
<a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord button.png" width="145"></a>
<a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a></a> Join Discord if you need help + ⭐ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐
</div>

To install Unsloth on your own computer, follow the installation instructions on our Github page [here](https://docs.unsloth.ai/get-started/installing-+-updating).

You will learn how to do [data prep](#Data), how to [train](#Train), how to [run the model](#Inference), & [how to save it](#Save)


### News

**Read our [blog post](https://unsloth.ai/blog/r1-reasoning) for guidance on how to train reasoning models.**

Visit our docs for all our [model uploads](https://docs.unsloth.ai/get-started/all-our-models) and [notebooks](https://docs.unsloth.ai/get-started/unsloth-notebooks).


### Installation

In [1]:
# %load_ext autoreload
# %autoreload 2

In [2]:
%%capture
# Skip restarting message in Colab
# import sys; modules = list(sys.modules.keys())
# for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None

# !pip install unsloth vllm
# !pip install --upgrade pillow
import importlib


### Unsloth

Use `PatchFastRL` before all functions to patch GRPO and other RL algorithms!

In [3]:
from unsloth import FastLanguageModel#, PatchFastRL
# PatchFastRL("GRPO", FastLanguageModel)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


Load up `Llama 3.1 8B Instruct`, and set parameters

In [4]:
from unsloth import is_bfloat16_supported
import torch
max_seq_length = 4096*2 # Can increase for longer reasoning traces
lora_rank = 64 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "meta-llama/meta-Llama-3.1-8B-Instruct",
    max_seq_length = max_seq_length,
    load_in_4bit = True, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.6, # Reduce if out of memory
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ], # Remove QKVO if out of memory
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth", # Enable long context finetuning
    random_state = 3407,
)

INFO 03-07 23:08:19 __init__.py:207] Automatically detected platform cuda.
==((====))==  Unsloth 2025.3.6: Fast Llama patching. Transformers: 4.49.0. vLLM: 0.7.3.
   \\   /|    NVIDIA H100 80GB HBM3. Num GPUs = 1. Max memory: 79.209 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.5.1+cu124. CUDA: 9.0. CUDA Toolkit: 12.4. Triton: 3.1.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: vLLM loading unsloth/meta-llama-3.1-8b-instruct-unsloth-bnb-4bit with actual GPU utilization = 59.55%
Unsloth: Your GPU has CUDA compute capability 9.0 with VRAM = 79.21 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 8192. Num Sequences = 226.
Unsloth: vLLM's KV Cache can use up to 40.83 GB. Also swap space = 6 GB.
INFO 03-07 23:08:26 config.py:549] This model supports multiple tasks: {'score', 'classify', 'rew

[W307 23:08:27.243863654 CUDAAllocatorConfig.h:28] Warning: expandable_segments not supported on this platform (function operator())


INFO 03-07 23:08:28 loader.py:1089] Loading weights with BitsAndBytes quantization.  May take a while ...
INFO 03-07 23:08:29 weight_utils.py:254] Using model weights format ['*.safetensors']


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 03-07 23:08:31 model_runner.py:1115] Loading model weights took 5.5976 GB
INFO 03-07 23:08:31 punica_selector.py:18] Using PunicaWrapperGPU.
INFO 03-07 23:08:33 worker.py:267] Memory profiling takes 1.89 seconds
INFO 03-07 23:08:33 worker.py:267] the current vLLM instance can use total_gpu_memory (79.21GiB) x gpu_memory_utilization (0.60) = 47.16GiB
INFO 03-07 23:08:33 worker.py:267] model weights take 5.60GiB; non_torch_memory takes 0.16GiB; PyTorch activation peak memory takes 1.13GiB; the rest of the memory reserved for KV Cache is 40.29GiB.
INFO 03-07 23:08:33 executor_base.py:111] # cuda blocks: 20625, # CPU blocks: 3072
INFO 03-07 23:08:33 executor_base.py:116] Maximum concurrency for 8192 tokens per request: 40.28x
INFO 03-07 23:08:37 model_runner.py:1434] Capturing cudagraphs for decoding. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI. If out-of-memory error 

Capturing CUDA graph shapes: 100%|██████████| 32/32 [00:13<00:00,  2.37it/s]

INFO 03-07 23:08:51 model_runner.py:1562] Graph capturing finished in 14 secs, took 0.83 GiB
INFO 03-07 23:08:51 llm_engine.py:436] init engine (profile, create kv cache, warmup model) took 19.69 seconds



Unsloth 2025.3.6 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


### Data Prep
<a name="Data"></a>

We directly leverage [@willccbb](https://gist.github.com/willccbb/4676755236bb08cab5f4e54a0475d6fb) for data prep and all reward functions. You are free to create your own!

In [5]:
import re
from datasets import load_dataset, Dataset
from search_module import search, get_question_answer, get_question_count, verify
from rl_helpers import get_qa_dataset

train_dataset, test_dataset = get_qa_dataset()

Loading FAISS index from: /root/reasoning/faiss_index
Successfully loaded FAISS index
Loading chunks from: /root/reasoning/saved_data/chunks.pkl
Loading questions from: /root/reasoning/saved_data/questions.json
Successfully loaded 341 chunks and 672 questions


<a name="Train"></a>
### Train the model

Now set up GRPO Trainer and all configurations!

In [6]:
import os
os.environ["WANDB_PROJECT"] = "bootstrap-search-rl"

In [7]:
# from UnslothGRPOTrainerTemp import UnslothGRPOConfig, _UnslothGRPOTrainer
import UnslothGRPOTrainerTemp
training_args = UnslothGRPOTrainerTemp.UnslothGRPOConfig(
    use_vllm = True, # use vLLM for fast inference!
    use_agentic_generate = True, # use agentic generation
    learning_rate = 5e-6,
    adam_beta1 = 0.9,
    adam_beta2 = 0.99,
    weight_decay = 0.1,
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "paged_adamw_8bit",
    logging_steps = 1,
    bf16 = is_bfloat16_supported(),
    fp16 = not is_bfloat16_supported(),
    per_device_train_batch_size = 16,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 8, # Decrease if out of memory
    max_prompt_length = 1024,
    max_completion_length = 1024,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 101,
    save_steps = 50,
    max_grad_norm = 0.1,
    report_to = "wandb", # Can use Weights & Biases
    output_dir = "full_local_training",

)

And let's run the trainer! If you scroll up, you'll see a table of rewards. The goal is to see the `reward` column increase!

You might have to wait 150 to 200 steps for any action. You'll probably get 0 reward for the first 100 steps. Please be patient!

| Step | Training Loss | reward    | reward_std | completion_length | kl       |
|------|---------------|-----------|------------|-------------------|----------|
| 1    | 0.000000      | 0.125000  | 0.000000   | 200.000000        | 0.000000 |
| 2    | 0.000000      | 0.072375  | 0.248112   | 200.000000        | 0.000000 |
| 3    | 0.000000      | -0.079000 | 0.163776   | 182.500000        | 0.000005 |


In [8]:


import rl_helpers
# importlib.reload(rl_helpers)


def agentic_generate(
        prompts:list[str],
        generate_fn,
        max_generations:int=6,
        ):
    return run_agent(generate_fn, tokenizer, prompts, max_generations)
model.agentic_generate = agentic_generate


from vllm import SamplingParams
verifier_sampling_params = SamplingParams(
    temperature = 0.1,
    top_p = 0.95,
    max_tokens = 4096,
)
def verifier_generate_fn(inputs):
    return model.fast_generate(
        inputs,
        sampling_params = verifier_sampling_params,
    )


run_agent = rl_helpers.run_agent
reward_correctness = rl_helpers.build_reward_correctness_fn(verifier_generate_fn, tokenizer,)
reward_formatting = rl_helpers.reward_formatting

import UnslothGRPOTrainerTemp
trainer = UnslothGRPOTrainerTemp.UnslothGRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        reward_correctness,
        reward_formatting,
    ],
    args = training_args,
    train_dataset = train_dataset,
)

In [9]:
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 604 | Num Epochs = 1 | Total steps = 200
O^O/ \_/ \    Batch size per device = 16 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (16 x 1 x 1) = 16
 "-____-"     Trainable parameters = 167,772,160/4,796,452,864 (3.50% trained)
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:wandb: WARNING If you're specifying your api key in code, ensu

['What method was used to obtain potable water for the crew?', 'What method was used to obtain potable water for the crew?', 'What method was used to obtain potable water for the crew?', 'What method was used to obtain potable water for the crew?', 'What method was used to obtain potable water for the crew?', 'What method was used to obtain potable water for the crew?', 'What method was used to obtain potable water for the crew?', 'What method was used to obtain potable water for the crew?', 'How long was the primary lithium hydroxide cartridge used for?', 'How long was the primary lithium hydroxide cartridge used for?', 'How long was the primary lithium hydroxide cartridge used for?', 'How long was the primary lithium hydroxide cartridge used for?', 'How long was the primary lithium hydroxide cartridge used for?', 'How long was the primary lithium hydroxide cartridge used for?', 'How long was the primary lithium hydroxide cartridge used for?', 'How long was the primary lithium hydroxi

/root/reasoning/UnslothGRPOTrainerTemp.py:943: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  completion_ids = [torch.tensor(ids, device=device) for ids in completion_ids]
Processed prompts: 100%|██████████| 16/16 [00:00<00:00, 19.46it/s, est. speed input: 4246.68 toks/s, output: 120.77 toks/s]


rewards_per_func: tensor([0.2500, 0.3937], device='cuda:0')


Step,Training Loss,reward,reward_std,completion_length,kl,rewards / reward_correctness,rewards / reward_formatting
1,0.000000,0.643750,0.565432,233.437500,0.000000,0.250000,0.393750
2,0.000000,0.431250,0.513990,143.562500,0.000000,0.125000,0.306250
3,0.000700,0.175000,0.324037,166.812500,0.017441,0.000000,0.175000


...

/root/reasoning/UnslothGRPOTrainerTemp.py:943: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  completion_ids = [torch.tensor(ids, device=device) for ids in completion_ids]
Processed prompts: 100%|██████████| 16/16 [00:00<00:00, 132.63it/s, est. speed input: 19328.48 toks/s, output: 265.79 toks/s]


rewards_per_func: tensor([0.0000, 0.1750], device='cuda:0')
['What was the maximum electric field perturbation recorded at site 7?', 'What was the maximum electric field perturbation recorded at site 7?', 'What was the maximum electric field perturbation recorded at site 7?', 'What was the maximum electric field perturbation recorded at site 7?', 'What was the maximum electric field perturbation recorded at site 7?', 'What was the maximum electric field perturbation recorded at site 7?', 'What was the maximum electric field perturbation recorded at site 7?', 'What was the maximum electric field perturbation recorded at site 7?', "What was the reason for the crew's difficulty in establishing the passive thermal control mode?", "What was the reason for the crew's difficulty in establishing the passive thermal control mode?", "What was the reason for the crew's difficulty in establishing the passive thermal control mode?", "What was the reason for the crew's difficulty in establishing the

Processed prompts: 100%|██████████| 16/16 [00:00<00:00, 62.12it/s, est. speed input: 10663.59 toks/s, output: 172.80 toks/s]


rewards_per_func: tensor([0.4375, 0.4375], device='cuda:0')
['What was the reason for the pronounced apparent relief seen in the cloud patterns in frames 8590 and 8591?', 'What was the reason for the pronounced apparent relief seen in the cloud patterns in frames 8590 and 8591?', 'What was the reason for the pronounced apparent relief seen in the cloud patterns in frames 8590 and 8591?', 'What was the reason for the pronounced apparent relief seen in the cloud patterns in frames 8590 and 8591?', 'What was the reason for the pronounced apparent relief seen in the cloud patterns in frames 8590 and 8591?', 'What was the reason for the pronounced apparent relief seen in the cloud patterns in frames 8590 and 8591?', 'What was the reason for the pronounced apparent relief seen in the cloud patterns in frames 8590 and 8591?', 'What was the reason for the pronounced apparent relief seen in the cloud patterns in frames 8590 and 8591?', 'What was the reason for the S-II boost engine cutoff being

KeyboardInterrupt: 

<a name="Inference"></a>
### Inference
Now let's try the model we just trained! First, let's first try the model without any GRPO trained:

In [19]:
from vllm import SamplingParams
import rl_helpers
sampling_params = SamplingParams(
    temperature = 0.5,
    top_p = 0.95,
    max_tokens = 4096,
)

def eval_generate_fn(inputs):
    return model.fast_generate(
        inputs,
        sampling_params = sampling_params,
        # lora_request = model.load_lora("full_local_training/checkpoint-200"),
    )


rl_helpers.run_eval(
    generate_fn=eval_generate_fn,
    verify_fn=reward_correctness,
    tokenizer=tokenizer,
)

Processed prompts: 100%|██████████| 68/68 [00:10<00:00,  6.43it/s, est. speed input: 1851.61 toks/s, output: 768.22 toks/s]


KeyboardInterrupt: 

In [ ]:
test_dataset[0]

Our reasoning model is much better - it's not always correct, since we only trained it for an hour or so - it'll be better if we extend the sequence length and train for longer!

<a name="Save"></a>
### Saving to float16 for VLLM

We also support saving to `float16` directly. Select `merged_16bit` for float16 or `merged_4bit` for int4. We also allow `lora` adapters as a fallback. Use `push_to_hub_merged` to upload to your Hugging Face account! You can go to https://huggingface.co/settings/tokens for your personal tokens.

In [ ]:
# Merge to 16bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_16bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_16bit", token = "")

# Merge to 4bit
if False: model.save_pretrained_merged("model", tokenizer, save_method = "merged_4bit",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "merged_4bit", token = "")

# Just LoRA adapters
if False: model.save_pretrained_merged("model", tokenizer, save_method = "lora",)
if False: model.push_to_hub_merged("hf/model", tokenizer, save_method = "lora", token = "")

### GGUF / llama.cpp Conversion
To save to `GGUF` / `llama.cpp`, we support it natively now! We clone `llama.cpp` and we default save it to `q8_0`. We allow all methods like `q4_k_m`. Use `save_pretrained_gguf` for local saving and `push_to_hub_gguf` for uploading to HF.

Some supported quant methods (full list on our [Wiki page](https://github.com/unslothai/unsloth/wiki#gguf-quantization-options)):
* `q8_0` - Fast conversion. High resource use, but generally acceptable.
* `q4_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q4_K.
* `q5_k_m` - Recommended. Uses Q6_K for half of the attention.wv and feed_forward.w2 tensors, else Q5_K.

[**NEW**] To finetune and auto export to Ollama, try our [Ollama notebook](https://colab.research.google.com/drive/1WZDi7APtQ9VsvOrQSSC5DDtxq159j8iZ?usp=sharing)

In [ ]:
# Save to 8bit Q8_0
if False: model.save_pretrained_gguf("model", tokenizer,)
# Remember to go to https://huggingface.co/settings/tokens for a token!
# And change hf to your username!
if False: model.push_to_hub_gguf("hf/model", tokenizer, token = "")

# Save to 16bit GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "f16")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "f16", token = "")

# Save to q4_k_m GGUF
if False: model.save_pretrained_gguf("model", tokenizer, quantization_method = "q4_k_m")
if False: model.push_to_hub_gguf("hf/model", tokenizer, quantization_method = "q4_k_m", token = "")

# Save to multiple GGUF options - much faster if you want multiple!
if False:
    model.push_to_hub_gguf(
        "hf/model", # Change hf to your username!
        tokenizer,
        quantization_method = ["q4_k_m", "q8_0", "q5_k_m",],
        token = "",
    )

Now, use the `model-unsloth.gguf` file or `model-unsloth-Q4_K_M.gguf` file in llama.cpp or a UI based system like Jan or Open WebUI. You can install Jan [here](https://github.com/janhq/jan) and Open WebUI [here](https://github.com/open-webui/open-webui)

And we're done! If you have any questions on Unsloth, we have a [Discord](https://discord.gg/unsloth) channel! If you find any bugs or want to keep updated with the latest LLM stuff, or need help, join projects etc, feel free to join our Discord!

Some other links:
1. Llama 3.2 Conversational notebook. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(1B_and_3B)-Conversational.ipynb)
2. Saving finetunes to Ollama. [Free notebook](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3_(8B)-Ollama.ipynb)
3. Llama 3.2 Vision finetuning - Radiography use case. [Free Colab](https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Llama3.2_(11B)-Vision.ipynb)
6. See notebooks for DPO, ORPO, Continued pretraining, conversational finetuning and more on our [documentation](https://docs.unsloth.ai/get-started/unsloth-notebooks)!

<div class="align-center">
  <a href="https://unsloth.ai"><img src="https://github.com/unslothai/unsloth/raw/main/images/unsloth%20new%20logo.png" width="115"></a>
  <a href="https://discord.gg/unsloth"><img src="https://github.com/unslothai/unsloth/raw/main/images/Discord.png" width="145"></a>
  <a href="https://docs.unsloth.ai/"><img src="https://github.com/unslothai/unsloth/blob/main/images/documentation%20green%20button.png?raw=true" width="125"></a>

  Join Discord if you need help + ⭐️ <i>Star us on <a href="https://github.com/unslothai/unsloth">Github</a> </i> ⭐️
</div>
